In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from bracketology_rl import Bracket
from bracketgym1game import TournamentEnv
import warnings
import gym
import numpy as np
from tourney_tools import * 
warnings.filterwarnings('ignore')
from IPython.display import display,clear_output

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_12328/2810803039.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

from stable_baselines3 import DQN,DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import ProgressBarCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement
from stable_baselines3.common.monitor import Monitor

In [3]:
import tensorboard

In [4]:
from pkg_resources import resource_filename
filepath = resource_filename('bracketology_rl', 'data/brackets_2022.json')
with open(filepath, 'r') as f:
    brackets_dict = json.load(f)

In [5]:
years = list(range(2003,2019))
year=np.random.choice(years)

bracket = Bracket(2022)
results=bracket.result


team_data=pd.read_csv('Process_data/pm_w_names.csv')


In [6]:
env=TournamentEnv(team_data=team_data,verbose=False,season=2022)


Reset! Sim 0 Bracket for year 2022:   0%|          | 0/120000 [00:00<?, ?it/s]

In [7]:
done=False
obs=env.reset()
obs

Reset! Sim 1 Bracket for year 2011:   0%|          | 1/120000 [00:00<2:26:44, 13.63it/s]

game: 0


array([ 0.        ,  1.3396323 , -0.2986242 , -0.64434147, -0.44361085,
        0.59799117,  0.32564294,  0.08042216, -0.22302395, -1.055259  ,
        0.36562905, -0.78026706, -0.7562189 , -0.6178707 , -1.2510682 ,
       -0.38069344,  0.        ,  0.17031893,  1.1564311 ,  1.1063443 ,
        0.1369939 ,  0.16413718, -0.5944668 ,  0.55392975,  0.79436874,
        0.09476367, -0.58890307,  0.25802216, -0.4634057 ,  1.2411865 ,
       -0.3804118 , -1.5884998 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.  

In [8]:
while not done:

    action=np.random.rand(env.action_space.shape[0])
    obs,score,done,info=env.step(action)
    print(info)


Reset! Sim 1 Bracket for year 2011:   0%|          | 34/120000 [00:00<11:07, 179.82it/s]

{'round': 1, 'game_num': 1, 'score': 1, 'total_score': 1}
{'round': 1, 'game_num': 2, 'score': 1, 'total_score': 2}
{'round': 1, 'game_num': 3, 'score': 1, 'total_score': 3}
{'round': 1, 'game_num': 4, 'score': 1, 'total_score': 4}
{'round': 1, 'game_num': 5, 'score': 1, 'total_score': 5}
{'round': 1, 'game_num': 6, 'score': 1, 'total_score': 6}
{'round': 1, 'game_num': 7, 'score': 1, 'total_score': 7}
{'round': 1, 'game_num': 8, 'score': 1, 'total_score': 8}
{'round': 1, 'game_num': 9, 'score': 1, 'total_score': 9}
{'round': 1, 'game_num': 10, 'score': 1, 'total_score': 10}
{'round': 1, 'game_num': 11, 'score': 1, 'total_score': 11}
{'round': 1, 'game_num': 12, 'score': 1, 'total_score': 12}
{'round': 1, 'game_num': 13, 'score': 1, 'total_score': 13}
{'round': 1, 'game_num': 14, 'score': 1, 'total_score': 14}
{'round': 1, 'game_num': 15, 'score': 1, 'total_score': 15}
{'round': 1, 'game_num': 16, 'score': 1, 'total_score': 16}
{'round': 1, 'game_num': 17, 'score': 1, 'total_score': 17

Reset! Sim 1 Bracket for year 2011:   0%|          | 64/120000 [00:00<08:51, 225.76it/s]

{'round': 5, 'game_num': 60, 'score': 0, 'total_score': 40}
{'round': 5, 'game_num': 61, 'score': 0, 'total_score': 40}
{'round': 6, 'game_num': 62, 'score': 0, 'total_score': 40}
{'round': 7, 'game_num': 63, 'score': 0, 'total_score': 40}


Reset! Sim 1 Bracket for year 2011:   0%|          | 64/120000 [00:20<08:51, 225.76it/s]

In [ ]:
## tensor board_log
tensorboard_log = "Logs/"
# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
# callback=ProgressBarCallback()
model = DDPG("MlpPolicy", env, 
                        action_noise=action_noise,
                        gamma=.99,
                        learning_rate=.01,
                        gradient_steps=-1, 
                        verbose=0,
                        tensorboard_log=tensorboard_log,
                        )
# model.predict(observation)

In [ ]:
obs=env.reset()
done=False
while not done:
    action=model.predict(obs)[0]

    obs,score, done,info=env.step(action)
    print(info)

In [ ]:
timesteps=50000000
try:
    model.learn(total_timesteps=timesteps, log_interval=10,reset_num_timesteps=True,tb_log_name='Tourney_bot',)
except KeyboardInterrupt:
    pass

In [ ]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), deterministic=False, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
273/6